In [ ]:
# importing the library modules

from datetime import datetime
import torch
from torch import nn
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from transformers import *
import random
from colorama import Fore, Back, Style
import matplotlib.pyplot as plt
import seaborn as sn
from seqeval.metrics import precision_score, recall_score, f1_score ,accuracy_score,classification_report
from seqeval.scheme import IOB2
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

### Defining Classes & Functions 

In [ ]:
# defining the specific format for saving the read data 

class TextGetter(object):
    
    def __init__(self,groupName,data):
        self.data = data
        self.groupName = groupName
        self.empty = False
        agg_func = lambda s: [(w, t,Id,Po,Se) for w, t , Id, Po, Se in zip(s["Word"].values.tolist(),
                                                     s["Tag"].values.tolist(),
                                                     s["Id"].values.tolist(),
                                                     s["Post #"].values.tolist(),s["Sentence #"].values.tolist())]
        self.grouped = self.data.groupby(self.groupName,sort=False).apply(agg_func)
        self.texts = [s for s in self.grouped]

In [ ]:
def dataset_summary(tag,B_NextI): 
    pred = ['Sum']*len(tag)
    y_actu = pd.Series(tag, name='Label')
    y_pred = pd.Series(pred, name='')
    df= pd.crosstab(y_actu, y_pred) 
    df = df.reset_index()
            
    if B_NextI == True:    
        df['Row'] = df['Label'].copy()
        for ii in range(len(df['Row'])):
            if (df['Row'][ii][:2] == 'B-') or (df['Row'][ii][:2] == 'I-'): 
                df['Row'][ii] = df['Row'][ii][2:]    
        for index in range(len(df)):
            if df.loc[index,'Row']=='O':
                df.loc[index,'Row']='zzzz'    
        
        df = df.sort_values(by = ['Row','Label'],axis=0)
        df = df.drop('Row',axis= 1)
    else:    
        for index in range(len(df)):
            if df.loc[index,'Label']=='O':
                df.loc[index,'Label']='zzzz' 
                
        df = df.sort_values(by = ['Label'],axis=0)
        
        for index in range(len(df)):
            if df.loc[index,'Label']=='zzzz':
                df.loc[index,'Label']='O' 
    
    Column_List = df.columns.to_list()
    Column_List_New = ['Label']
    for lab in TagLabel :
        if lab in Column_List:
            Column_List_New.extend([lab])
    if len(Column_List_New)==1:
        Column_List_New = Column_List_New+ ['Sum']
       
    if B_NextI == True:        
        Column_List_New = ['Label']
        for lab in TagLabel :
            if lab in Column_List:
                Column_List_New.extend([lab])
        if len(Column_List_New)==1:
            Column_List_New = Column_List_New+ ['Sum']
        Column_List = Column_List_New
          
    df = df.reindex( columns=Column_List)          
    return df

In [ ]:
# function for making confusion matrix with and without "O" label

def Make_Confusion_Graph_Tabel(target,pred,Labels,Prefix,Font_Scale,ShowPercent,Sum_O_Zero): 

    if ShowPercent:
        aa = confusion_matrix(target,pred,labels= Labels,normalize= 'true')
        aa = 100.0 * aa
        Fmt = '.1f'        
    else:
        aa = confusion_matrix(target,pred,labels= Labels)
        Fmt = 'd'
        
    if Sum_O_Zero:
        aa[len(aa[:,0])-1,len(aa[:,0])-1] = 0
        
    if Prefix:                          
        df_cm = pd.DataFrame(aa, index = [i for i in Labels], columns = [i for i in Labels]) 
    else:
        df_cm = pd.DataFrame(aa, index = [i[2:] if i != 'O' else i for i in Labels], columns = [i[2:] if i != 'O' else i for i in Labels ]) 

    FigSize =(len(Labels)* 0.8 ,len(Labels)*0.4)
    plt.figure(figsize = FigSize)
    sn.set(font_scale=Font_Scale) # for label size
    sn.heatmap(df_cm, annot=True,cmap='Blues', cbar=False, annot_kws={"size": 14},fmt = Fmt)
    plt.show()
    return df_cm

In [ ]:
#tokenize the input reviews and organize their proper labels (assign "O" to not important tokens)

def tokenize_and_preserve_labels(getter_Texts):
    Tokenized_Texts =[[]]*len(getter_Texts)
    Labels =[[]]*len(getter_Texts)
    
    for ii,sent in enumerate(getter_Texts):
        Tokenized_Texts[ii] =['[CLS]']
        Labels[ii] =['O']
        token =[]
        lab = []
        for s in sent:
            tokenized_s = tokenizer.tokenize(s[0])
            labe_s = s[1]
            label =[] 
            label = [labe_s]*len(tokenized_s)
            token.extend(tokenized_s)        
            lab.extend(label)    
        Tokenized_Texts[ii].extend(token)
        Tokenized_Texts[ii].extend(['[SEP]'])
        
        Labels[ii].extend(lab)
        Labels[ii].extend(['O'])
    return Tokenized_Texts,Labels           


In [ ]:
# convert tokenize and their related labels to index

def ids_and_mask(text_tokenize,Labels):
    text_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in text_tokenize],value=0,
                          maxlen=seq_length, dtype="long", truncating="post", padding="post")
    
    tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in Labels], maxlen=seq_length,
                           value=-100, dtype="long", truncating="post", padding="post") 
                          
    text_mask = [[int(i>0) for i in ii] for ii in text_ids]
    
    return text_ids,tags,text_mask

In [ ]:
# function for plotting the loss and accuracy values

def plot_loss_accuracy(df_score,xlim_from,xlim_to,yLow_loss,yHigh_loss,yLow_accuracy,flag_title):
    sn.set(context="notebook", style="white", palette=None,
               font="sans-serif", font_scale=1.1, color_codes=True, rc=None)
    df_plot = df_score.copy()
    plt.figure(figsize = (16,4))

    # Plot negative loss function
    plt.subplot(1, 2, 1)
    plt.xlim(xlim_from,xlim_to)
    
    
    df_plot.loc[0,'Tr_loss']=1000
    df_plot.loc[0,'Va_loss']=1000

    
    Tr_loss, = plt.plot(df_plot.Tr_loss)
    Va_loss, = plt.plot(df_plot.Va_loss)
    
    plt.xlabel('epoch')
    plt.ylabel('Loos')
    plt.yscale('log')
    plt.ylim(yLow_loss,yHigh_loss)
    plt.grid(ls='--')
    if flag_title == 1:
        plt.title('Loss - train ends at %.2f | val at %.2f' %(df_plot.tail(1).Tr_loss, df_plot.tail(1).Va_loss))
    else:
        plt.title('Loss (Training & Validation)')
        
    val_xmin_loss = np.argmin(np.array(df_plot.Va_loss)) 
    val_ymin_loss = min(df_plot.Va_loss)
    aaa = plt.scatter(val_xmin_loss, val_ymin_loss, marker='o', color='black',lw=2) # plot point    
    
    plt.legend([Tr_loss, Va_loss,aaa], ['Training', 'Validation','Valid Min'])
    
    
    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.xlim(xlim_from,xlim_to)
    Tr_accuracy, = plt.plot(df_plot.Tr_accur)
    Va_accuracy, = plt.plot(df_plot.Va_accur)
    plt.xlabel('epoch')
    plt.ylabel('accuracy (%)')
    plt.ylim(yLow_accuracy,101)
    plt.grid(ls='--')
    if flag_title == 1:
        plt.title('Accuracy - train ends at %.2f%% | val at %.2f%%' %(df_plot.tail(1).Tr_accur , df_plot.tail(1).Va_accur ))
    else:
        plt.title('Accuracy (Training & Validation)')

    val_xmax_accu = np.argmax(np.array(df_plot.Va_accur))    
    val_ymax_accu = max(df_plot.Va_accur)
    
    aaa = plt.scatter(val_xmax_accu, val_ymax_accu, marker='o', color='black',lw=2) # plot point    
    plt.legend([Tr_accuracy, Va_accuracy,aaa], ['Training', 'Validation','Valid Max'])
    global fig
    fig = plt.gcf()
    plt.show(block= False)
    
    if flag_title == 1:
        print("Min Valid Loss: %.4f     in Epoch: %.f" %(val_ymin_loss,val_xmin_loss))
        print("Max Valid Accu: %.2f %%    in Epoch: %.f" %(val_ymax_accu,val_xmax_accu)) 
        print("____________________________________________________________")
    return (True) 

In [ ]:
# function for plotting the F1_score value

def plot_F1_Score(df_score,xlim_from,xlim_to,yLow_F1,flag_title): 
    sn.set(context="notebook", style="white", palette=None,
               font="sans-serif", font_scale=1.1, color_codes=True, rc=None)
    plt.figure(figsize = (16,4))

    plt.subplot(1, 2, 2)
    plt.xlim(xlim_from,xlim_to)
    train_F1Score, = plt.plot(df_score.Tr_F1)
    valid_F1Score, = plt.plot(df_score.Va_F1)
    plt.xlabel('epoch')
    plt.ylabel('f1-score')
    plt.ylim(yLow_F1,1.05)
    plt.grid(ls='--')
    if flag_title == 1:
        plt.title('f1-score - train ends at %.2f | val at %.2f' %(df_score.tail(1).Tr_F1 , df_score.tail(1).Va_F1 ))
    else:
        plt.title('f1-score (Training & Validation)')
        
        
    valid_xmax_F1 = np.argmax(np.array(df_score.Va_F1))          
    valid_ymax_F1 = max(df_score.Va_F1)
    aaa = plt.scatter(valid_xmax_F1, valid_ymax_F1, marker='o', color='black',lw=2) # plot point    
    plt.legend([train_F1Score, valid_F1Score,aaa], ['Training', 'Validation','Valid Max'])     
              
    train_ymax_F1 = max(df_score.Tr_F1)
    train_xmax_F1 = np.argmax(np.array(df_score.Tr_F1))    
    
    global fig
    fig = plt.gcf()
    plt.show(block= False)
    if flag_title == 1:
        print("Training   Max f1-score: %.4f     in Epoch: %.f" %(train_ymax_F1,train_xmax_F1))
        print("Validation Max f1-score: %.4f     in Epoch: %.f" %(valid_ymax_F1,valid_xmax_F1))
    print("____________________________________________________________")
    return (True)

In [ ]:
# building the model (concatenation BERT 4 last layers + BiLSTM)

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.embedding = model_B
        
        self.linear512_numLabel = nn.Linear(512, num_labels)
        self.embedding_dropout = nn.Dropout(0.3)         
        self.lstm3072_256 = nn.LSTM(3072, 256,bidirectional=True)
        
    def forward(self, inputs_data,mask): 
        with torch.no_grad():
            word_emb, sent_emb,hidden_layers = self.embedding(inputs_data,
                                                              attention_mask= mask)  
            
        x = torch.cat([hidden_layers[11], hidden_layers[10],hidden_layers[9], hidden_layers[8]], dim=2)
        x = self.embedding_dropout(x)
               
        x, _ = self.lstm3072_256(x)
        
        x = self.linear512_numLabel(x)  
           
        return x             #[BatchSize, seq_length, numLabel]

### End (Defining Classes & Functions)

In [ ]:
# describing and initializing the random seed (the random seed is used to get the same random number in each 
# repetition of codes)

seed = 3054
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.
torch.manual_seed(seed)

In [ ]:
# select CPU or GPU to run the program. 
# n_gpu: 0 used "CPU" & n_gpu: 1 used "GPU"

if torch.cuda.is_available():
    torch.device("cuda") 
    n_gpu = torch.cuda.device_count()
    print("torch.device : cuda")
    print(torch.cuda.get_device_name(0))
else:    
    torch.device("cpu")
    n_gpu = 0;
    print("torch.device : cpu")
print(n_gpu)    

In [ ]:
# reading and loading the dataset6

data = pd.read_csv("./Data/Dokhtara/Dokhtara_Word_Dataset_Main.csv", encoding="utf-8").fillna(method="ffill")
getterTexts = TextGetter('Post #',data).texts

data.head(10)

In [ ]:
# setting the percentage of splitting the dataset

valid_size_real = 0.15
test_size = 0.30
valid_size = valid_size_real /(1- test_size)

In [ ]:
# shuffling and splitting the train, valid and test dataset

valid_size_real = 0.15
test_size = 0.30
valid_size = valid_size_real /(1- test_size)
Random_State = 2018

print('number text:',len(getterTexts),'\n')

trainValid_getter, test_getter = train_test_split(getterTexts, random_state=Random_State, test_size=test_size)
train_getter, valid_getter = train_test_split(trainValid_getter,random_state=Random_State, test_size=valid_size)

print('number train       :',len(train_getter))
print('number valid       :',len(valid_getter))
print('number test        :',len(test_getter))

In [ ]:
# initializing some hyperparameters

seq_length = 120
BatchSize = 16
LRate = 0.0001   # LRate =3e-5     0.001

In [ ]:
# assigning a unique index to each label

tags_name = list(set(g[1] for gett in getterTexts for g in gett))
tags_name.sort()
tag2idx = {t: i for i, t in enumerate(tags_name)}
num_labels = len(tags_name)

TagLabel =tags_name.copy()
TagLabel.remove('O')   # without "O"

print("num_labels :",num_labels)
print("")
print("Tags name  :",tags_name)
print("")
print("tag2idx    :",tag2idx)

In [ ]:
# a brief report of the distribution of the label in the dataset 

BNextI = True
AllTags = [g[1] for gett in getterTexts for g in gett]
report_dataSet = dataset_summary(AllTags, B_NextI= BNextI)

TrTags = [g[1] for gett in train_getter for g in gett]
report_Train = dataset_summary(TrTags, B_NextI= BNextI)

VaTags = [g[1] for gett in valid_getter for g in gett]
report_Valid = dataset_summary(VaTags, B_NextI= BNextI)

TsTags = [g[1] for gett in test_getter for g in gett]
report_Test = dataset_summary(TsTags, B_NextI= BNextI)

res = pd.DataFrame({'Label':tags_name})
res['Train'] = 0
res['Valid'] = 0
res['Test'] = 0
res['Sum'] = [0]*len(res)
for ii in range(len(res)):
    
    for jj in range(len(report_Train)):
        if res['Label'][ii] == report_Train['Label'][jj]:
            res['Train'][ii]  = report_Train['Sum'][jj]
            break           
    
    for jj in range(len(report_Valid)):
        if res['Label'][ii] == report_Valid['Label'][jj]:
            res['Valid'][ii]  = report_Valid['Sum'][jj]
            break   
            
    if len(test_getter) != 0.0 :
        for jj in range(len(report_Test)):
            if res['Label'][ii] == report_Test['Label'][jj]:
                res['Test'][ii]  = report_Test['Sum'][jj]
                break
            
    for jj in range(len(report_dataSet)):
        if res['Label'][ii] == report_dataSet['Label'][jj]:
            res['Sum'][ii]  = report_dataSet['Sum'][jj]
            break
            
print('*** Data Set ***\n\n')
print('train (num text)  :',len(train_getter))
print('valid (num text)  :',len(valid_getter))
print('test  (num text)  :',len(test_getter))
print('')
print(res)


In [ ]:
# assigning a unique index to each label

weights = 'bert-base-uncased'
model_B = BertModel.from_pretrained(weights,output_hidden_states = True,num_labels = num_labels)
tokenizer = BertTokenizer.from_pretrained(weights, do_lower_case=True, sep_token='[SEP]'
                                                 , pad_token='[PAD]', cls_token='[CLS]', mask_token='[MASK]')#PAD

In [ ]:
# tokenizing and assigning unique indices to tokenized reviews and their related true labels


train_tokenized,train_labels = tokenize_and_preserve_labels(train_getter) 
train_ids,tr_tags,train_masks = ids_and_mask(train_tokenized,train_labels)

valid_tokenized,valid_labels = tokenize_and_preserve_labels(valid_getter)
valid_ids,val_tags,valid_masks = ids_and_mask(valid_tokenized,valid_labels)

test_tokenized,test_labels = tokenize_and_preserve_labels(test_getter) 
test_ids,ts_tags,test_masks = ids_and_mask(test_tokenized,test_labels)


In [ ]:
# a brief report of the distribution of the label in the dataset after BERT tokenization 

print('### After Tokenized ###')
BNextI = True

TrTags = [tags_name[l_i] for i,l in enumerate(tr_tags) for ii,l_i in enumerate(l) if train_ids[i][ii] != 0]
report_Train = dataset_summary(TrTags , B_NextI= BNextI)

VaTags = [tags_name[l_i] for i,l in enumerate(val_tags) for ii,l_i in enumerate(l) if valid_ids[i][ii] != 0]
report_Valid = dataset_summary(VaTags , B_NextI= BNextI)

TsTags = [tags_name[l_i] for i,l in enumerate(ts_tags) for ii,l_i in enumerate(l) if test_ids[i][ii] != 0]   
report_Test = dataset_summary(TsTags , B_NextI= BNextI)

res = pd.DataFrame({'Label':TagLabel})
res['Train'] = 0
res['Valid'] = 0
res['Test'] = 0
res['Sum'] = [0] *len(res)
for ii in range(len(res)):
    
    for jj in range(len(report_Train)):
        if res['Label'][ii] == report_Train['Label'][jj]:
            res['Train'][ii]  = report_Train['Sum'][jj]
            break           
    
    for jj in range(len(report_Valid)):
        if res['Label'][ii] == report_Valid['Label'][jj]:
            res['Valid'][ii]  = report_Valid['Sum'][jj]
            break   
            
    if len(test_getter) != 0.0 :
        for jj in range(len(report_Test)):
            if res['Label'][ii] == report_Test['Label'][jj]:
                res['Test'][ii]  = report_Test['Sum'][jj]
                break
            
    res['Sum'][ii]  = res['Train'][ii]+res['Valid'][ii]+res['Test'][ii]
            

print('*** Data Set ***\n')
print(res)
#Make_Confusion_Graph_Tabel(res,Font_Scale = 1.4,ShowPercent= False, Sum_O_Zero= False)

In [ ]:
# create loder objects for train dataset , validation dataset & test dataset

#------------- trainloader------------------------
x_tr = torch.tensor(train_ids, dtype=torch.long)
y_tr = torch.tensor(tr_tags, dtype=torch.float32)
m_tr = torch.tensor(train_masks, dtype=torch.int32)

train_data = TensorDataset(x_tr, y_tr, m_tr)
train_sampler = RandomSampler(train_data)
trainloader = DataLoader(train_data, sampler=train_sampler, batch_size=BatchSize)

#------------- validloader------------------------
x_val = torch.tensor(valid_ids, dtype=torch.long)
y_val = torch.tensor(val_tags, dtype=torch.float32)
m_val = torch.tensor(valid_masks, dtype=torch.int32)

valid_data = TensorDataset(x_val, y_val, m_val)
valid_sampler = SequentialSampler(valid_data)
validloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=BatchSize)


#------------- testloader------------------------
x_test = torch.tensor(test_ids, dtype=torch.long)
y_test = torch.tensor(ts_tags, dtype=torch.float32)
m_test = torch.tensor(test_masks, dtype=torch.int32)

test_data = TensorDataset(x_test, y_test, m_test)
testloader = DataLoader(test_data, batch_size=BatchSize)


In [ ]:
# loading the defined model and check to load in CPU or GPU 

model = Model()

if n_gpu == 1:
    model.cuda()
else:     
    model.cpu()  

In [ ]:
# defining the loss function

loss_function = nn.CrossEntropyLoss(reduction='elementwise_mean',size_average=True, ignore_index=-100)  # LogSoftmax + ClassNLL Loss

if n_gpu == 1:
    loss_function = loss_function.cuda()
       

In [ ]:
# defining the optimizer function and setting its parameters

FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(
                  optimizer_grouped_parameters,
                  lr=LRate,
                  eps=1e-8
                  )

In [ ]:
## Training Model 

In [ ]:
##### Training  process#####

startA = datetime.now()
print("Now Time : ",startA)


show_confusion_after_any_epoch = True
show_fclassification_report_after_any_epoch = True
Digits = 4

validat_loss = 0
min_valid_loss = 1000
df_score = pd.DataFrame()

max_valid_accuracy = 0
max_valid_F1Score = 0

yLow_loss = 0.001   # for Change
yHigh_loss = 2.0    # for Change
yLow_accuracy = 0  # for Change
yLow_f1_score = 0.0  # for Change

Tr_f1Score,Tr_RecallScore,Tr_PreciScore,Tr_Accure =[],[],[],[]
Va_f1Score,Va_RecallScore,Va_PreciScore,Va_Accure =[],[],[],[]

min_valid_loss_epoch = 0
max_valid_F1Score_epoch = 0

epoch = 1
Average = 'micro'   
mode_score = 'strict' 
Flag_Traning_Countinue = True
Epochs_Max = 60
#for epoch in range(1, Epochs_Max+1):
while Flag_Traning_Countinue == True:
    start = datetime.now()
    print ("Epoch :", epoch,"                             ")
    
    #------------ train part ------------------------
    train_loss, valid_loss = [], []
    predictions = []
    true_labels = []  
    data2 =[]
    ii = 0
    model.train()
    for data,target, mask in trainloader:
        
        optimizer.zero_grad()
        
        target = torch.tensor(target, dtype=torch.long)
        
        if n_gpu == 1:
            data = data.to('cuda')
            target = target.to('cuda')
            mask = mask.to('cuda')  
            
        output= model(data,mask)
        loss = loss_function(output.flatten(start_dim=0, end_dim=1),target.flatten())
        
        ##  backward propagation
        loss.backward()

        ##  weight optimization
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=1.0)
        optimizer.step()
        
        train_loss.append(loss.item())
        
        label_ids = target.to('cpu').numpy()
        true_labels.extend(label_ids)
        output2 = output.detach().cpu().numpy()
        
        data_tmp = data.to('cpu').numpy()
        data2.extend(data_tmp)

        predictions.extend([list(p) for p in np.argmax(output2, axis=2)]) 
        ii +=1
        print("train batch:  %.f / %.f " %(ii,len(train_ids)/BatchSize),end='\r')
    print("                                ",end='\r')
    training_loss = np.mean(train_loss)
    
    train_pred_tags = [[tags_name[l_i] for ii,l_i in enumerate(l)  if data2[i][ii] != 0 and data2[i][ii] != 101 and data2[i][ii] != 102] for i,l in enumerate(predictions)]
    train_tags = [[tags_name[l_i] for ii,l_i in enumerate(l)  if data2[i][ii] != 0 and data2[i][ii] != 101 and data2[i][ii] != 102] for i,l in enumerate(true_labels)]
    
    Train_Acc = accuracy_score(train_tags, train_pred_tags)*100.0
    train_F1Score = f1_score(train_tags, train_pred_tags,average=Average,scheme=IOB2, mode= mode_score )
    train_precision_score = precision_score(train_tags, train_pred_tags,average=Average,scheme=IOB2, mode= mode_score)
    train_recall_score = recall_score(train_tags, train_pred_tags,average=Average,scheme=IOB2, mode= mode_score)

    ## --------------------evaluation part -------------------------
    model.eval()
    predictions = []
    true_labels = []
    ii = 0
    data2 =[]
    for data, target, mask in validloader:
        target = torch.tensor(target, dtype=torch.long)
        if n_gpu == 1:
            data = data.to('cuda')
            target = target.to('cuda')
            mask = mask.to('cuda')
            
        with torch.no_grad():
            output= model(data,mask)

        loss = loss_function(output.flatten(start_dim=0, end_dim=1),target.flatten())
        
        
        valid_loss.append(loss.item())
        
        label_ids = target.to('cpu').numpy()
        true_labels.extend(label_ids)
        output2 = output.detach().cpu().numpy()
        
        data_tmp = data.to('cpu').numpy()
        data2.extend(data_tmp)
        
        predictions.extend([list(p) for p in np.argmax(output2, axis=2)]) 
        ii += 1
        print("valid batch:  %.f / %.f " %(ii,len(valid_ids)/BatchSize),end='\r')


    validat_loss = np.mean(valid_loss) 
    
    valid_pred_tags = [[tags_name[l_i] for ii,l_i in enumerate(l)  if data2[i][ii] != 0 and data2[i][ii] != 101 and data2[i][ii] != 102] for i,l in enumerate(predictions)]
    valid_tags = [[tags_name[l_i] for ii,l_i in enumerate(l)  if data2[i][ii] != 0 and data2[i][ii] != 101 and data2[i][ii] != 102] for i,l in enumerate(true_labels)]

    
    Valid_Acc = accuracy_score(valid_tags,valid_pred_tags)*100.0
    valid_F1Score = f1_score(valid_tags,valid_pred_tags,average=Average,scheme=IOB2, mode= mode_score )
    valid_precision_score = precision_score(valid_tags,valid_pred_tags,average=Average,scheme=IOB2, mode= mode_score)
    valid_recall_score = recall_score(valid_tags,valid_pred_tags,average=Average,scheme=IOB2, mode= mode_score)
    
    #----------- Svae model in min Loss Validation------------------
    
    if validat_loss < min_valid_loss :
        torch.save(model,'./model/Valid_Min_Loss_Model.pt')
        min_valid_loss = validat_loss
        min_valid_loss_epoch = epoch
    
    # ------------------- Report & Graph -------------------------
    if epoch == 1:
        temp = pd.DataFrame({
                             'Tr_accur':[0],
                             'Va_accur':[0],
                             'Tr_loss' : [0],
                             'Va_loss' : [0],
                             'Tr_F1':[0],
                             'Va_F1':[0],
                             'Tr_precis':[0],  
                             'Va_precis':[0],
                             'Tr_recall':[0],  
                             'Va_recall':[0]
                             }) 
        df_score = df_score.append(temp, ignore_index = True) 
    temp = pd.DataFrame({
                             'Tr_accur':[Train_Acc],
                             'Va_accur':[Valid_Acc],
                             'Tr_loss' : [training_loss],
                             'Va_loss' : [validat_loss],
                             'Tr_F1':[train_F1Score],  
                             'Va_F1':[valid_F1Score],
                             'Tr_precis':[train_precision_score],  
                             'Va_precis':[valid_precision_score],
                             'Tr_recall':[train_recall_score],  
                             'Va_recall':[valid_recall_score]
                             })    
    df_score = df_score.append(temp, ignore_index = True)
     
    print (Fore.BLUE +"            Accuracy |     Loss    |   Precision |    Recall   |   F1-score  |"+Fore.RESET)
    print('Training  :  %.2f %% |    %.4f   |    %.4f   |    %.4f   |    %.4f   |'
           %(Train_Acc,training_loss,train_precision_score,train_recall_score,train_F1Score))
    print('Validation:  %.2f %% |    %.4f   |    %.4f   |    %.4f   |    %.4f   |'
           %(Valid_Acc,validat_loss,valid_precision_score,valid_recall_score,valid_F1Score))


    plot_loss_accuracy(df_score,0, Epochs_Max, yLow_loss,yHigh_loss, yLow_accuracy, 1)
    plot_F1_Score(df_score,0, Epochs_Max, yLow_f1_score, 1)
      
    if show_confusion_after_any_epoch :
        print('\nTraining------------------------------------------------------------------------------------')
        Make_Confusion_Graph_Tabel(sum(train_tags,[]), sum(train_pred_tags,[]), Labels= TagLabel, Font_Scale = 1.4,
                                                                    Prefix = True, ShowPercent= True, Sum_O_Zero= True)
        print('\n\nValidation----------------------------------------------------------------------------------')
        Make_Confusion_Graph_Tabel(sum(valid_tags,[]), sum(valid_pred_tags,[]), Labels= TagLabel, Font_Scale = 1.4,
                                                                    Prefix = True, ShowPercent= True, Sum_O_Zero= True)
        print('--------------------------------------------------------------------------------------------')
        
    if show_fclassification_report_after_any_epoch:
        print('\nTraining------------------------------------------------------------------------------------')
        print(classification_report(train_tags, train_pred_tags, digits = Digits,scheme=IOB2, mode= mode_score))

        print('\n\nValidation----------------------------------------------------------------------------------')
        print(classification_report(valid_tags,valid_pred_tags, digits = Digits,scheme=IOB2, mode= mode_score))
        print('-----------------------------------------------------------------------------------------------')
       
    end = datetime.now()
    print("")
    print("Time : ", (end - start))
    
    print("________________________________________________________________________________________________________")
    
#------------------------- Trainning Countine Condition Check ---------------------
    if epoch - min_valid_loss_epoch > 10 :
        Flag_Traning_Countinue = False
        Epochs_Max = epoch
    else:    
        epoch += 1             
#----------------------------------------------------------------------------------        
plot_loss_accuracy(df_score,0, Epochs_Max, yLow_loss,yHigh_loss, yLow_accuracy, 1)
plot_F1_Score(df_score,0, Epochs_Max, yLow_f1_score, 1)        


print("Sum Times : ", (end - startA))

## Test Nodel (with Test Dataset)

In [ ]:
# defining the test function using the model that saved in minimum validation loss and 
# applying it to the test dataset for prediction

def select_model(model):
    if n_gpu == 1:
        model.cuda()
    else:     
        model.cpu()
    predictions = []
    true_labels = []
    data2 =[]
    true_labels_rep = []
    test_loss = []
    jj = 0
    allPred =[]
    outputs_test = []
    
    model.eval()
    for data, target,mask in testloader:
    
        target = torch.tensor(target, dtype=torch.long)
        if n_gpu == 1:
            data = data.to('cuda')
            target = target.to('cuda')
            mask = mask.to('cuda')  
        with torch.no_grad():
            output= model(data,mask)
        loss = loss_function(output.flatten(start_dim=0, end_dim=1),target.flatten())
        
        test_loss.append(loss.item())
    

        label_ids = target.to('cpu').numpy()

        true_labels.extend(label_ids)
        true_labels_rep.append(label_ids)
        
        output2 = output.detach().cpu().numpy()
        
        data_tmp = data.to('cpu').numpy()
        data2.extend(data_tmp)
        
        predictions.extend([list(p) for p in np.argmax(output2, axis=2)])
        
        outputs_test.extend(output.detach().cpu().tolist())
    
        jj += 1
        print("Test batch:  %.f / %.f " %(jj,len(x_test)/BatchSize),end='\r')
        
        
    outputs_test = torch.tensor(outputs_test, dtype=torch.float32).sigmoid()
    outputs_test = np.array(outputs_test)    
  
    Test_Loss= np.mean(test_loss)

    test_pred_tags = [[tags_name[l_i] for ii,l_i in enumerate(l)  if data2[i][ii] != 0 and data2[i][ii] != 101 and data2[i][ii] != 102] for i,l in enumerate(predictions)]
    test_tags = [[tags_name[l_i] for ii,l_i in enumerate(l)  if data2[i][ii] != 0 and data2[i][ii] != 101 and data2[i][ii] != 102] for i,l in enumerate(true_labels)]

    Test_Acc = accuracy_score(test_tags,test_pred_tags)*100.0
    test_F1Score = f1_score(test_tags,test_pred_tags,average=Average,scheme=IOB2, mode= mode_score )
    test_precision_score = precision_score(test_tags,test_pred_tags,average=Average,scheme=IOB2, mode= mode_score)
    test_recall_score = recall_score(test_tags,test_pred_tags,average=Average,scheme=IOB2, mode= mode_score)

    print (Fore.BLUE +"       Accuracy |     Loss    |   Precision |    Recall   |   F1-score  |"+Fore.RESET)
    print('Test :  %.2f %% |    %.4f   |    %.4f   |    %.4f   |    %.4f   |'
       %(Test_Acc,Test_Loss,test_precision_score,test_recall_score,test_F1Score))
    return test_tags,test_pred_tags,outputs_test

model.cpu()
model=torch.load('./model/Valid_Min_Loss_Model.pt')
print('Test Model Parameter: Valid Minimum Loss (epoch=',min_valid_loss_epoch,')\n')
test_tags,test_pred_tags,outputs = select_model(model)
print("=========================================================================")


## Test Reports

In [ ]:
# test dataset all reports (classification report, confusion matrix, and some predicted reviews sample)

print('\n*** Test Model Parameter: Valid Minimum Loss (epoch=',min_valid_loss_epoch, ') ***\n')

prediction = test_pred_tags.copy()
output = outputs.copy()
    
OutsAll = []
for ii in range(len(test_tags)):
    OutsLine = []
    for jj in range(len(test_tags[ii])):
        OutsLine.append(output[ii,jj+1,:])
    OutsAll.extend([OutsLine])    
    
#group_sel = group_name.replace('#',':')
TestId, TestPo, TestSe = [], [], []
TestWords, TestTags, TestPreds, TestChecks = [], [], [], []

for sent in test_getter: 
    Id, Po, Se = [], [], []
    Words, Tags, Preds, Checks = [], [], [], []
    for ii,s in enumerate(sent):
        Words.extend([s[0]])        
        Tags.extend([s[1]]) 
        Preds.extend([''])
        Checks.extend([''])
        if ii == 0 :
            Id.extend([s[2]])
            Po.extend([s[3]])
            Se.extend([s[4]])
        else:
            Id.extend([''])
            Po.extend([''])
            Se.extend([''])
        
    TestWords.extend([Words])
    TestTags.extend([Tags])
    TestId.extend([Id])
    TestPo.extend([Po])
    TestSe.extend([Se])
    TestPreds.extend([Preds])
    TestChecks.extend([Checks]) 
    

for ii in range(len(TestWords)):

    kk = 0   
    
    for jj in range(len(TestWords[ii])):
        try:
            TestPreds[ii][jj] = prediction[ii][kk]
            kk += len(tokenizer.tokenize(TestWords[ii][jj])) 
            if TestTags[ii][jj] == TestPreds[ii][jj]:
                TestChecks[ii][jj] = 'True'  
            else:
                TestChecks[ii][jj] = 'False'    
        except:
            break
for ii in range(len(TestPreds)-1,0,-1) :
    for jj in range(len(TestPreds[ii])-1,0,-1):
        if TestPreds[ii][jj] == '':
            del TestId[ii][jj]
            del TestPo[ii][jj]
            del TestSe[ii][jj]
            del TestWords[ii][jj]
            del TestTags[ii][jj]
            del TestPreds[ii][jj]
            del TestChecks[ii][jj]
            
 #Svae Test Named Entities Result --------------------------------------------------------
df_predict= pd.DataFrame({'Id':sum(TestId,[]),'Post #':sum(TestPo,[]), 'Sentence #':sum(TestSe,[]),
                          'Word':sum(TestWords,[]),
                          'Tag':sum(TestTags,[]),'Predict': sum(TestPreds,[]),'Check':sum(TestChecks,[])})

df_predict.to_csv("./Result/Test_Dataset_Name_Entities.csv",sep=",", index=None)
#-----------------------------------------------------------------------------------------


Test_Acc = accuracy_score(TestTags,TestPreds)*100.0
test_F1Score = f1_score(TestTags,TestPreds,average=Average,scheme=IOB2, mode= mode_score )
test_precision_score = precision_score(TestTags,TestPreds,average=Average,scheme=IOB2, mode= mode_score)
test_recall_score = recall_score(TestTags,TestPreds,average=Average,scheme=IOB2, mode= mode_score)

print ('===========================================================')
print (Fore.BLUE +"       Accuracy |  Precision  |    Recall   |   F1-score  |"+Fore.RESET)
print('Test :  %.2f %% |    %.4f   |    %.4f   |    %.4f   |'
   %(Test_Acc,test_precision_score,test_recall_score,test_F1Score))
print ('===========================================================')

#Prints-----------------------------------------------------------------------------------
from seqeval.metrics import  classification_report
print('----------------------------------------------------------------------------')
print('mode: strict')
result_report = classification_report(TestTags,TestPreds, digits = Digits,scheme=IOB2, mode= 'strict')
print(result_report)
print('----------------------------------------------------------------------------')
print('library: sklearn')
from sklearn.metrics import  classification_report
result_report = classification_report(sum(TestTags,[]),sum(TestPreds,[]), digits = Digits,labels =TagLabel)
print(result_report)
from seqeval.metrics import  classification_report

print('---------------------------------------------------------------------------\n')

Make_Confusion_Graph_Tabel(sum(TestTags,[]),sum(TestPreds,[]), Labels= tags_name, Font_Scale = 1.4,
                                                                    Prefix = True, ShowPercent= True, Sum_O_Zero= True)
Make_Confusion_Graph_Tabel(sum(TestTags,[]),sum(TestPreds,[]), Labels= tags_name, Font_Scale = 1.4,
                                                                    Prefix = True, ShowPercent= False, Sum_O_Zero= True)
df_predict.head(10)

## End Test